# Sensor Management Example

In [21]:
import numpy as np
import random
from ordered_set import OrderedSet
from datetime import datetime, timedelta
from mpar_sim.radar import PhasedArrayRadar


start_time = datetime.now()

from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState

## Generate Ground Truths

In [22]:
# np.random.seed(1990)
# random.seed(1990)

# Generate transition model
transition_model = CombinedLinearGaussianTransitionModel([
    ConstantVelocity(1),
    ConstantVelocity(1),
    ConstantVelocity(0),
])

yps = range(0, 100, 10)  # y value for prior state
truths = OrderedSet()
ntruths = 1
time_max = 500

xstart = 3000
xdirection = 10
ystart = 0
ydirection = 1
zstart = 0
zdirection = 0

# Generate ground truths
truth = GroundTruthPath([
    GroundTruthState([xstart, xdirection, ystart, ydirection, zstart, zdirection],
                     timestamp=start_time)])
# TODO: Create a radar ground truth class with an RCS field
truth.rcs = 10
for j in range(1, time_max):
  current_truth = GroundTruthState(transition_model.function(truth[j-1], noise=True, time_interval=timedelta(seconds=0.1)),
                                   timestamp=start_time + timedelta(seconds=j*0.1))
  current_truth.rcs = 10
  truth.append(current_truth)

truths.add(truth)

# truth = GroundTruthPath([
#     GroundTruthState([xstart, xdirection, ystart, -ydirection, zstart, zdirection],
#                      timestamp=start_time)])
# # TODO: Create a radar ground truth class with an RCS field
# truth.rcs = 10
# for j in range(1, time_max):
#   current_truth = GroundTruthState(transition_model.function(truth[j-1], noise=True, time_interval=timedelta(seconds=0.1)),
#                                    timestamp=start_time + timedelta(seconds=j*0.1))
#   current_truth.rcs = 10
#   truth.append(current_truth)

# truths.add(truth)



0

Plot ground truths using the Plotterly class

In [23]:
from stonesoup.plotter import Plotterly

plotter = Plotterly()
plotter.plot_ground_truths(truths, [0, 2])
plotter.fig

## Sensor setup

## Tracker Setup

Create the predictor and updater

In [24]:
from stonesoup.predictor.kalman import KalmanPredictor

predictor = KalmanPredictor(transition_model)

from stonesoup.updater.kalman import ExtendedKalmanUpdater
# Measurement model is added to detections by the sensor
updater = ExtendedKalmanUpdater(measurement_model=None)


Initialize a prior for each target.

**TODO: This should be done when a new detection is made**

In [25]:
from stonesoup.types.state import GaussianState

priors = []
# xdirection = 1.2*100
# ydirection = 1.2
for j in range(ntruths):
  priors.append(GaussianState([xstart, xdirection, ystart, ydirection, zstart, zdirection],
                              np.diag([0.5, 0.5, 0.5, 0.5, 0, 0]),
                              timestamp=start_time))


Initialize tracks from each prior

In [26]:
from stonesoup.types.track import Track

tracks = {Track([prior]) for prior in priors}

Create a hypothesizer and data associator

In [27]:
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis
hypothesiser = DistanceHypothesiser(
    predictor, updater, measure=Mahalanobis(), missed_distance=5)

from stonesoup.dataassociator.neighbour import NearestNeighbour
data_associator = NearestNeighbour(hypothesiser)

## 

In [28]:
from mpar_sim.radar import PhasedArrayRadar
from stonesoup.sensor.radar import AESARadar
from stonesoup.types.state import StateVector

sensor = PhasedArrayRadar(
  position=np.array([0, 0, 0]),
  rotation_offset=StateVector(np.array([0, 0, 0])),
  position_mapping=(0, 2, 4),
)
sensor.timestamp = start_time

## Run the Simulation

In [29]:
# Generate list of timesteps from ground truth timestamps
from mpar_sim.beam.common import aperture2beamwidth
from mpar_sim.look import RadarLook
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track


timesteps = []
for state in truths[0]:
  timesteps.append(state.timestamp)

# Select action(s)
look = RadarLook(
    start_time=start_time,
    azimuth_beamwidth=5,
    elevation_beamwidth=0.1,
    tx_power=10e3,
    azimuth_steering_angle=0,
    elevation_steering_angle=0,
    bandwidth=100e6,
    pulsewidth=10e-6,
    prf=5e3,
    n_pulses=100,
)
# Schedule looks
sensor.load_look(look)
all_measurements = []
i = 0
for timestep in timesteps[1:]:

  # Collect measurements for the current timestep
  measurements = set()
  measurements |= sensor.measure(OrderedSet(
      truth[timestep] for truth in truths), noise=True)
  all_measurements.append(measurements)

  # Associate measurements to tracks
  hypotheses = data_associator.associate(tracks, measurements, timestep)
  # Update tracks
  for track in tracks:
    hypothesis = hypotheses[track]
    if hypothesis.measurement:
      post = updater.update(hypothesis)
      track.append(post)

      look.azimuth_steering_angle=sensor.measurement_model.function(truths[0].states[i+1], noise=False)[1].degrees
      sensor.load_look(look)
    else:
      track.append(hypothesis.prediction)
  i += 1


Plot the ground truth, tracks, and uncertainty ellipses for each target

In [30]:
plotter = Plotterly()
plotter.plot_sensors(sensor)
plotter.plot_ground_truths(truths, [0, 2])
plotter.plot_measurements(all_measurements, [0, 2])
plotter.plot_tracks(tracks, [0, 2], uncertainty=False)
plotter.fig